# Imports

In [ ]:
import sys
from collections import Counter
import itertools
import os

import matplotlib as mpl
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

try:
    import batteryanalytics
except ModuleNotFoundError as ie:
    sys.path.append( os.path.join(os.path.abspath(""), "../") )
    import batteryanalytics
from batteryanalytics import utils as ba_utils
from batteryanalytics.utils import CNNModelKey as ModelKey

from IPython.core.display import display, HTML, Image

# Parameters

In [ ]:
basename = "mechanical_loading_data.csv.gz"
dirname = "../data"

target = "Temperature [C]"

results_dirname = "../cnn_results/cnn_models"

# Function Definitions

In [ ]:
def get_best_model_lookup(models):
    best_model_lookup = dict()
    for model_key,model in models.items():
        fields_dict = model_key._asdict()
        fields_dict["model_id"] = "best"
        best_model_key = ModelKey(**fields_dict)
        res_model_key = best_model_lookup.setdefault(best_model_key, model_key)
        if models[model_key].validate_loss[-1] < models[res_model_key].validate_loss[-1]:
            best_model_lookup[best_model_key] = model_key
    return best_model_lookup

In [ ]:
def get_expected_performance(samples, target, predictions):
    expected_performance_df = dict()
    for model_key,prediction_mapping in predictions.items():
        holdouts = model_key.holdouts
        fields = [
            field
            for field in model_key._fields
            if field not in ["holdouts", "model_id"]
        ]
        assert len(holdouts) == 1, f"expecting 1 holdout but found {len(holdouts)}"

        offset = model_key.window + model_key.horizon - 1
        y_true = samples[holdouts[0]].loc[:,pd.IndexSlice[:,:,target]].values.squeeze()
        y_pred = prediction_mapping[holdouts[0]]
        
        expected_performance_df.setdefault(
            tuple(getattr(model_key, field) for field in fields),
            list()
        ).extend(
            np.square(y_true[offset:] - y_pred[offset:]).tolist()
        )

    expected_performance_df = pd.DataFrame(
        [
            key + (len(value), np.min(value), np.max(value), np.mean(value), np.std(value), np.median(value))
            for key,value in expected_performance_df.items()
        ],
        columns=fields + ["count", "min", "max", "mean", "std", "median"]
    )
    expected_performance_df.sort_values(
        by=list(expected_performance_df.columns), inplace=True
    )
    expected_performance_df.reset_index(
        drop=True, inplace=True
    )
    return expected_performance_df

In [ ]:
def get_model_groupings(model_keys, grouping_fields):
    grouping_dict = dict()
    for model_key in model_keys:
        fields_dict = model_key._asdict()
        for field in grouping_fields:
            fields_dict[field] = "N/A"
        general_model_key = ModelKey(**fields_dict)
        grouping_dict.setdefault(general_model_key, list()).append(model_key)
    return grouping_dict

In [ ]:
def input_predictions(sample, sample_df, predictions, model_keys, model_labels=None):
    if model_labels is None:
        model_labels = list(map(str,model_keys))
    assert len(model_keys) == len(model_labels), f"length of model_keys ({len(model_keys)}) and model_labels ({len(model_labels)}) does not match"
    for model_key,model_label in zip(model_keys,model_labels):
        values = predictions[model_key][sample]
        values = np.hstack([
            [np.nan]*(len(sample_df.index)-len(values)),
            values
        ])
        sample_df[model_label] = values
    return sample_df

In [ ]:
def display_model_groupings(sample, sample_df, target, predictions, best_model_lookup, grouping_fields, filename=None):
    model_groupings = get_model_groupings(best_model_lookup.keys(), grouping_fields)
    model_groupings = { 
        model_key:list(map(best_model_lookup.get,model_grouping))
        for model_key,model_grouping in model_groupings.items()
        if model_key.holdouts[0] == sample
    }   
    
    n_cols = 3 
    n_rows = int(np.ceil(len(model_groupings)/n_cols))
    fig = plt.figure(constrained_layout=True, figsize=(8*n_cols,5*n_rows))
#     fig.set_constrained_layout_pads(w_pad=1.5)
    gs = mpl.gridspec.GridSpec(
        nrows=n_rows,
        ncols=n_cols,
        figure=fig
    )   
    
    for ii,(general_model_key,model_group) in enumerate(sorted(model_groupings.items())):
        title = "\n".join(
            f"{field}={getattr(general_model_key,field)}"
            for field in ModelKey._fields
            if field not in ["holdouts", "model_id"] + list(grouping_fields)
        )   
    
        ax = fig.add_subplot(gs[ii])
        ax.plot(sample_df.loc[:,pd.IndexSlice[:,:,target]], color="black", label=target)
        tmp_sample_df = input_predictions(
            sample,
            pd.DataFrame(index=sample_df.index),
            predictions,
            model_group,
            model_labels=[
                ", ".join([
                    str(getattr(specific_model_key, field))
                    for field in ModelKey._fields
                    if field in grouping_fields
                ])  
                for specific_model_key in model_group
            ]   
        )   
        ba_utils.display_df_columns(tmp_sample_df, title=title, title_loc="left", ax=ax)
        cur_col = ii % n_cols
        cur_row = int(ii / n_cols)
        if cur_col == 1 and cur_row+1 == n_rows:
            ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.07), ncol=1)
#     handles, labels = ax.get_legend_handles_labels() 
#     fig.legend(handles, labels, loc="upper center", bbox_to_anchor=(0.5, -0.07), ncol=1)
    if filename is not None:
        plt.savefig(filename)
    plt.show()

# Data Loading

In [ ]:
filename = os.path.join(dirname, basename)
raw_df = pd.read_csv(filename, header=[0,1,2], index_col=0, compression="gzip")
raw_df.info(verbose=True)
with pd.option_context("display.max_rows", 10, "display.max_columns", None):
    display(raw_df)
    display(raw_df.describe())

# Data Preprocessing

In [ ]:
data_df = raw_df.copy()

levels_0 = list()
levels_1 = list()
for column in data_df.columns:
    if column[0] not in levels_0:
        levels_0.append(column[0])
    if column[1] not in levels_1:
        levels_1.append(column[1])

samples = {
    pair:data_df.xs(pair, axis="columns", level=(0,1), drop_level=False).dropna()
    for pair in itertools.product(levels_0, levels_1)
}
summary1_df = pd.DataFrame(
    [[
            sample_df.index[-1]-sample_df.index[0],
            len(sample_df.index)
    ] for sample_df in samples.values() ],
    columns=["time span", "samples"]
).describe()

for sample_df in samples.values():
    series = sample_df.loc[:,pd.IndexSlice[:,:,target]].copy()
    idx = np.squeeze(series.values).argmax()
    sample_df.drop(sample_df.index[2*idx+1:], inplace=True)
#     vmin, vmax = series.min(), series.max()
#     delta = vmax - vmin
#     idx = series > 0.05*delta + vmin
#     cut_off = np.squeeze(idx.iloc[::-1].idxmax().values).item()
#     sample_df.drop(sample_df.index[sample_df.index > cut_off], inplace=True)

summary2_df = pd.DataFrame(
    [[
            sample_df.index[-1]-sample_df.index[0],
            len(sample_df.index)
    ] for sample_df in samples.values() ],
    columns=["time span", "samples"]
).describe()

with pd.option_context("display.max_rows", 10, "display.max_columns", None):
    display(summary1_df)
    display(summary2_df)
    display(data_df)

## Data Visualization

In [ ]:
def get_unique(items):
    seen = set()
    unique = list()
    for item in items:
        if item not in seen:
            unique.append(item)
            seen.add(item)
    return unique        
level0 = get_unique(raw_df.columns.get_level_values(0))
level1 = get_unique(raw_df.columns.get_level_values(1))
level2 = get_unique(raw_df.columns.get_level_values(2))

for c0 in level0:
    display(HTML(f"<h1>{c0}</h1>"))
    
#     if not c0.startswith("500"):
#         continue
    for c1 in level1:
        display(HTML(f"<h2>{c1}</h2>"))
#         if not c1.startswith("20"):
#             continue
        sample_df = samples[(c0,c1)]
        
        n_cols = 3
        n_rows = int(np.ceil(len(level2)/n_cols))
#         fig = plt.figure(constrained_layout=True, figsize=(n_cols*8,5*n_rows))
        fig = plt.figure(constrained_layout=True, figsize=(16,8))
        gs = mpl.gridspec.GridSpec(
            nrows=n_rows,
            ncols=n_cols,
            figure=fig
        )
        for ii,c2 in enumerate(level2):
            ax = fig.add_subplot(gs[ii])
            ax.plot(sample_df[(c0,c1,c2)], zorder=1)
            ax.plot(raw_df[(c0,c1,c2)], zorder=0)
            ax.set_xlim(raw_df.index[0], raw_df.index[-1])
            ax.set_title(c2)
            ax.set_xlabel("Time")
        fig.suptitle("{}, {}".format(c0,c1))
        plt.show()
    break

# Load Models

In [ ]:
feature_mapping = dict()
for feature in set(data_df.columns.get_level_values(2)):
    feature_mapping[ba_utils.sanitize_feature_name(feature)] = feature
    
time_function = ba_utils.time_function()
models, predictions = time_function(ba_utils.load_cnn_models)(results_dirname, feature_mapping)

## Loaded Model Info

In [ ]:
model_keys = set(models.keys())
prediction_keys = set(predictions.keys())
print(f"{len(model_keys):7d} Models")
print(f"{len(prediction_keys):7d} Predictions")
print()
print(f"Missing Models:      {len(prediction_keys-model_keys)}")
print(f"Missing Predictions: {len(model_keys-prediction_keys)}")
print()
for field in ModelKey._fields:
    counts = Counter(getattr(model_key, field) for model_key in prediction_keys)
    print(f"{len(counts):6d} {field}")
    
best_model_lookup = get_best_model_lookup(models)

# Performance Calculations

In [ ]:
expected_performance_df = get_expected_performance(samples, target, predictions)
with pd.option_context("display.max_rows", 10, "display.max_columns", None):
    display(expected_performance_df)

# Performance Plots

## Overall Performance

In [ ]:
tmp_df = expected_performance_df.set_index([
    "variables",
    "window",
    "horizon",
    "dim",
    "training_filter",
    "n_epochs"
]).sort_index()

tmp_dirname = os.path.join(os.path.dirname(results_dirname), "plots")
tmp_filename = os.path.join(tmp_dirname, "performance.png")
ba_utils.mkdirs(tmp_dirname)
ba_utils.display_bar_plot(
    tmp_df[["mean"]].transpose(),
    tmp_df[["std"]].rename(columns={"std":"mean"}).transpose(),
    ax_title="Overall Performance",
    ncol=1,
    filename=tmp_filename
)

## Performance by `dim`

In [ ]:
print(ModelKey._fields)
tmp_df = pd.pivot_table(
    expected_performance_df,
    values="mean",
    index=["variables", "window", "horizon", "training_filter"],
    columns=["dim"],
    aggfunc=np.sum
).sort_index()
tmp_yerr_df = pd.pivot_table(
    expected_performance_df,
    values="std",
    index=["variables", "window", "horizon", "training_filter"],
    columns=["dim"],
    aggfunc=np.sum
).sort_index()

tmp_dirname = os.path.join(os.path.dirname(results_dirname), "plots/dim")
tmp_filename = os.path.join(tmp_dirname, "performance.png")
ba_utils.mkdirs(tmp_dirname)
ba_utils.display_bar_plot(
    tmp_df,
    tmp_yerr_df.rename(columns={"std":"mean"}),
    ax_title="Performance by dim",
    label_rotation=25,
    label_alignment="right",
    ncol=len(tmp_df.columns),
    filename=tmp_filename
)

for sample,sample_df in samples.items():
    display(HTML(f"<h1>{sample}</h1>"))
    tmp_filename = os.path.join(
        tmp_dirname,
        "-".join(map(
            lambda item: "_".join(item.split()),
            sample
        )) + ".png"
    )
    print(tmp_filename)
    display_model_groupings(
        sample,
        sample_df,
        target,
        predictions,
        get_best_model_lookup(models),
        ["dim"],
        filename=tmp_filename
    )
    break

## Performance by `training_filter`

In [ ]:
tmp_df = pd.pivot_table(
    expected_performance_df,
    values="mean",
    index=["variables", "window", "horizon", "dim"],
    columns=["training_filter"],
    aggfunc=np.sum
)
tmp_yerr_df = pd.pivot_table(
    expected_performance_df,
    values="std",
    index=["variables", "window", "horizon", "dim"],
    columns=["training_filter"],
    aggfunc=np.sum
)

tmp_dirname = os.path.join(os.path.dirname(results_dirname), "plots/training_filter")
tmp_filename = os.path.join(tmp_dirname, "performance.png")
ba_utils.mkdirs(tmp_dirname)
ba_utils.display_bar_plot(
    tmp_df,
    tmp_yerr_df.rename(columns={"std":"mean"}),
    ax_title="Performance by training_filter",
    label_rotation=25,
    label_alignment="right",
    ncol=len(tmp_df.columns),
    filename=tmp_filename
)

for sample,sample_df in samples.items():
    display(HTML(f"<h1>{sample}</h1>"))
    tmp_filename = os.path.join(
        tmp_dirname,
        "-".join(map(
            lambda item: "_".join(item.split()),
            sample
        )) + ".png"
    )
    print(tmp_filename)
    display_model_groupings(
        sample,
        sample_df,
        target,
        predictions,
        get_best_model_lookup(models),
        ["training_filter"],
        filename=tmp_filename
    )

## Performance by `variables`

In [ ]:
tmp_df = pd.pivot_table(
    expected_performance_df,
    values="mean",
    index=["window", "horizon", "dim", "training_filter"],
    columns=["variables"],
    aggfunc=np.sum
).sort_index()
tmp_yerr_df = pd.pivot_table(
    expected_performance_df,
    values="std",
    index=["window", "horizon", "dim", "training_filter"],
    columns=["variables"],
    aggfunc=np.sum
).sort_index()

tmp_dirname = os.path.join(os.path.dirname(results_dirname), "plots/variables")
tmp_filename = os.path.join(tmp_dirname, "performance.png")
ba_utils.mkdirs(tmp_dirname)
ba_utils.display_bar_plot(
    tmp_df,
    tmp_yerr_df.rename(columns={"std":"mean"}),
    ax_title="Performance by variables",
    label_rotation=25,
    label_alignment="right",
    ncol=1,
    filename=tmp_filename
)

for sample,sample_df in samples.items():
    display(HTML(f"<h1>{sample}</h1>"))
    tmp_filename = os.path.join(
        tmp_dirname,
        "-".join(map(
            lambda item: "_".join(item.split()),
            sample
        )) + ".png"
    )
    print(tmp_filename)
    display_model_groupings(
        sample,
        sample_df,
        target,
        predictions,
        get_best_model_lookup(models),
        ["variables"],
        filename=tmp_filename
    )

# Video

In [ ]:
def make_video(sample, sample_df, target, model_keys, grouping_fileds, predictions):
    assert all(model_keys[0].variables == model_key.variables for model_key in model_keys)
    assert all(model_keys[0].window == model_key.window for model_key in model_keys)
    assert all(model_keys[0].horizon == model_key.horizon for model_key in model_keys)
    
    features = list(model_keys[0].variables)
    window = model_keys[0].window
    horizon = model_keys[0].horizon
    colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
    
    lines = list()
    points = list()
    spans = list()
    
    n_cols = len(features)
    n_rows = 3
    fig = plt.figure(constrained_layout=True, figsize=(16,10))
    gs = mpl.gridspec.GridSpec(
        nrows=n_rows,
        ncols=n_cols,
        figure=fig
    )
    
    ax = fig.add_subplot(gs[:-1,:])
    target_line, = ax.plot(sample_df.loc[:,pd.IndexSlice[:,:,target]], color="black", label=target)
    for ii,model_key in enumerate(model_keys):
        color = ["r", "g", "b"][ii % 3]
        values = predictions[model_key][sample]
        label = "; ".join([
            f"{field}={getattr(model_key, field)}"
            for field in grouping_fileds
        ])
        lines.append(
            ax.plot(sample_df.index[-len(values):], values, color=color)[0]
        )
        points.append(
            ax.scatter(sample_df.index[-len(values)], values[0], color=color, label=label)
        )
    spans.append(
        ax.axvspan(sample_df.index[0], sample_df.index[window], color="gray", alpha=0.25)
    )
    handles, labels = ax.get_legend_handles_labels() 
    title = "\n".join([
        f"{field}={getattr(model_keys[0],field)}"
        for field in model_keys[0]._fields
        if field not in grouping_fileds
    ])
    ax.set_title(title, loc="left")
    
    for ii,feature in enumerate(features):
        color = colors[ii % len(colors)]
        ax = fig.add_subplot(gs[-1,ii])
        ax.plot(sample_df.loc[:,pd.IndexSlice[:,:,feature]], color=color)
        spans.append(
            ax.axvspan(sample_df.index[0], sample_df.index[window], color=color, alpha=0.25)
        )
        ax.set_title(feature)
    fig.legend(handles, labels, loc="upper center", bbox_to_anchor=(0.5, -0.07), ncol=1)
    plt.show()
#     return
    
    def init():
        for line in lines:
            line.set_data([], [])
        for point in points:
            point.set_offsets(np.empty(shape=(0,2)))
        for span in spans:
            xy = span.get_xy()
            xy[[0,1,4],0] = sample_df.index[0]
            xy[[2,3],0] = sample_df.index[window]
            span.set_xy(xy)
        legend = fig.legend(handles, labels, loc="upper center", bbox_to_anchor=(0.5, -0.07), ncol=1)
        return lines + points + spans + [legend]

    def animate(frame):
        print(frame, end=" ")
        
        target_line.set_data(
            sample_df.index[:frame+window],
            sample_df.iloc[:frame+window,:].loc[:,pd.IndexSlice[:,:,target]]
        )
        for line,point,model_key in zip(lines,points,model_keys):
            values = predictions[model_key][sample]
            assert len(values) == len(sample_df.index)
            line.set_data(sample_df.index[:frame+window+horizon], values[:frame+window+horizon])
            point.set_offsets([[sample_df.index[frame+window+horizon-1], values[frame+window+horizon-1]]])
        for span in spans:
            xy = span.get_xy()
            xy[[0,1,4],0] = sample_df.index[frame]
            xy[[2,3],0] = sample_df.index[frame+window]
            span.set_xy(xy)
        legend = fig.legend(handles, labels, loc="upper center", bbox_to_anchor=(0.5, -0.07), ncol=1)
        return [target_line] + lines + points + spans + [legend]
    
    #frames = np.arange(len(sample_df.index)-window-horizon+1)
    argmax = np.argmax(sample_df.loc[:,pd.IndexSlice[:,:,target]].values)
    frames= np.arange(argmax-5*window, argmax+2*window)
    frames = frames[::int(np.ceil(len(frames)/60))]
    print(len(frames))
    anim = mpl.animation.FuncAnimation(
        fig,
        animate,
        init_func=init,
        frames=frames,
        interval=100,
        blit=True,
    )
#     anim.save("test.gif", writer="imagemagick")
    display(HTML(anim.to_jshtml()))

grouping_fileds = ["training_filter"]
model_groupings = get_model_groupings(best_model_lookup.keys(), grouping_fileds)
for ii,(sample,sample_df) in enumerate(samples.items()):
    display(HTML(f"<h1>{sample}</h1>"))
    sample_model_groupings = {
        model_key:list(map(best_model_lookup.get,model_grouping))
        for model_key,model_grouping in model_groupings.items()
        if model_key.holdouts[0] == sample
    }
    for general_model_key,specific_model_keys in sample_model_groupings.items():
        make_video(sample, sample_df, target, specific_model_keys, grouping_fileds, predictions)
        break
    break